In [ ]:
# We need PyTorch and GPyTorch
import torch
from torch import Tensor
import gpytorch
# Gaussian processes really don't work well without 64-bit precision
torch.set_default_dtype(torch.float64)

from norm_constrain import NormConstrainedKernel, NormConstrainedMean

# And a few more players
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

In [ ]:
# Import the data, check normalization, which should be 2 * (2\pi**2)

with open('nk_afdmc.dat', 'r') as fd:
    lines = fd.readlines()

x = [] ; y = [] ; err = []
for line in lines:
    buf = line.split()
    x.append(float(buf[0]))
    y.append(float(buf[1]))
    err.append(float(buf[2]))
x = np.array(x) ; y = np.array(y) ; err = np.array(err)

dx = x[1]-x[0]
ig = (y * x**2).sum() *dx / (2 * np.pi**2)
print(ig)


In [ ]:
# Subsample training data
subrate = 10
train_x = torch.tensor(x[::subrate])
train_y = torch.tensor(y[::subrate])
train_err = torch.tensor(err[::subrate])


# Normalization value
nval = 4 * np.pi**2

# Create the GPyTorch model
class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, nval):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)

        
        # Using the normalization-constrained model
        self.covar_module = NormConstrainedKernel(norm_value=nval)
        self.mean_module = NormConstrainedMean(norm_value=nval)
        
 
    def forward(self, x):
        self.mean_module.sigma = self.covar_module.sigma
        self.mean_module.gamma = self.covar_module.gamma
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

likelihood = gpytorch.likelihoods.FixedNoiseGaussianLikelihood(noise=train_err)

model = GPRegressionModel(train_x, train_y, likelihood, nval)


In [ ]:
# Training
training_iter = 100

model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    print(output)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    print(loss.shape)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   sigma: %.3f   gamma: %.3f' % (
        i + 1, training_iter, loss.item(),
        model.covar_module.sigma.item(),
        model.covar_module.gamma.item()
    ))
    optimizer.step()